## Initialisierung

In [ ]:
# Packages installieren (ggf. beim ersten Lauf aktivieren, danach auskommentieren)
#install.packages("tidyverse")
#install.packages("leaflet")
#install.packages("corrplot")

# Packages laden
library(readr)
library(dplyr)
library(tidyr)
library(lubridate)
library(leaflet)
library(corrplot)
library(ggplot2)

# Konfiguration
base_url = "https://raw.githubusercontent.com/miracum/c19ss2021/master/uebung-5/"

## Data Prep

### Patientenstammdaten laden

In [ ]:
# Patientenstammdaten Bielefeld laden
patients.bielefeld <- read_csv(paste(base_url, "rohdaten/bielefeld/csv/patients.csv", sep=""), col_types = cols(BIRTHDATE = col_date(format = "%Y-%m-%d")))
head(patients.bielefeld)

### Geovisualisierung der Patientenstammdaten

In [ ]:
# Geovisualisierung der Bielefelder Patientenstammdaten
leaflet() %>%
  setView(10.217850, 51.151509, zoom=6) %>%
  addTiles() %>%
  addMarkers(data = patients.bielefeld, ~LON, ~LAT, clusterOptions = markerClusterOptions())

### Histogramm nach Geburtsjahr

In [ ]:
# Bielefelder Patientenstämme nach Geburtsjahr & Geschlecht plotten
ggplot(data=patients.bielefeld, aes(x=lubridate::year(BIRTHDATE), fill=GENDER)) +
    geom_histogram(binwidth=5)

### Alter berechnen & Histogramm nach Alter

In [ ]:
# Alter der Bielefelder Patienten berechnen & plotten
patients.bielefeld <- mutate(patients.bielefeld, AGE = case_when(is.na(DEATHDATE) ~ as.period(interval(start = BIRTHDATE, end = Sys.Date()))$year, TRUE ~ as.period(interval(start = BIRTHDATE, end = DEATHDATE))$year))
ggplot(data=patients.bielefeld, aes(x=AGE, fill=GENDER)) +
    geom_histogram(binwidth=5)

### Diagnosen laden

In [ ]:
# Diagnosen Bielefeld laden
diagnoses.bielefeld <- read_csv(paste(base_url, "rohdaten/bielefeld/csv/conditions.csv", sep=""), col_types = cols(START = col_date(format = "%Y-%m-%d"), STOP = col_date(format = "%Y-%m-%d")))
head(diagnoses.bielefeld)

### Exploration der Diagnosen

In [ ]:
# Diagnosen der Bielefelder Kohorte
ggplot(data=diagnoses.bielefeld, aes(x=DESCRIPTION)) +
    geom_bar()

### Top-10 Diagnosen

In [ ]:
# Barplot der Top 10 Diagnosen der Bielefelder Kohorte
ggplot(data=diagnoses.bielefeld %>% 
            group_by(DESCRIPTION) %>% 
            summarize(FREQ=n()) %>%
            slice_max(FREQ, n=10), 
       aes(x=reorder(DESCRIPTION, -FREQ), y=FREQ)) +
    geom_bar(stat="identity") +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

## Auswertung der Daten

### Patient:innen mit Covid-19-Diagnosen auswählen

In [ ]:
# Bielefelder Patienten mit COVID-19 selektieren
cohort.patients.bielefeld <- diagnoses.bielefeld %>% 
    filter(DESCRIPTION == "COVID-19") %>% 
    select(PATIENT) %>% distinct()
head(cohort.patients.bielefeld)

### Ergänzung demographische Angaben

In [ ]:
# Demographische Angaben ergänzen
cohort.patients.bielefeld <- cohort.patients.bielefeld %>% 
    inner_join(patients.bielefeld, by=c("PATIENT" = "Id")) %>%
    select(PATIENT, BIRTHDATE, DEATHDATE, GENDER, AGE, CITY)
head(cohort.patients.bielefeld)

### Ableitung Vitalstatus aus Todesdatum

In [ ]:
# Vitalstatus ergänzen
cohort.patients.bielefeld <- mutate(cohort.patients.bielefeld, VITALSTATUS = case_when(is.na(DEATHDATE) ~ "ALIVE", TRUE ~ "DEAD"))
head(cohort.patients.bielefeld)

### Boxplot Alter vs. Vitalstatus

In [ ]:
# Alter gegen Vitalstatus plotten
ggplot(cohort.patients.bielefeld, aes(x=VITALSTATUS, y=AGE)) +
  geom_boxplot()

### Selektion der Begleitdiagnosen der Kohorte

In [ ]:
# Begleitdiagnosen der Bielefelder Patienten bestimmen
cohort.secondary_diagnoses.bielefeld <- cohort.patients.bielefeld %>% 
            inner_join(diagnoses.bielefeld, by="PATIENT") %>%
            filter(!(DESCRIPTION %in% c("COVID-19", "Suspected COVID-19")))
head(cohort.secondary_diagnoses.bielefeld)

### Top-10 Begleitdiagnosen der Kohorte

In [ ]:
# Barplot der Top 10 Begleitdiagnosen der Bielefelder COVID-19-Patienten
ggplot(data=cohort.secondary_diagnoses.bielefeld %>%
            group_by(DESCRIPTION) %>% 
            summarize(FREQ=n()) %>%
            slice_max(FREQ, n=10),
       aes(x=reorder(DESCRIPTION, -FREQ), y=FREQ)) +
    geom_bar(stat="identity") +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

### Reduktion auf Nebendiagnosen Adipositas, Erschöpfung & Atemnot

In [ ]:
# Zu untersuchende Nebendiagnosen (Adipositas, Erschöpfung und Atemnot) auslesen
tmp <- cohort.secondary_diagnoses.bielefeld %>% 
       filter(DESCRIPTION %in% c("Body mass index 30+ - obesity (finding)","Fatigue (finding)","Respiratory distress (finding)")) %>%
       select(PATIENT, DESCRIPTION) %>%
       mutate(DIAGNOSISPRESENT = 1)
head(tmp)

# Diagnosen zu Spalten pivotieren
tmp <- pivot_wider(tmp, names_from=DESCRIPTION, values_from=DIAGNOSISPRESENT, values_fill=NA)
head(tmp)

# Spalten umbenennen
colnames(tmp) <- c("PATIENT", "OBESITY", "FATIGUE", "RESPIRATORY_DISTRESS")
head(tmp)

# Stammdaten & zu untersuchende Diagnosen zusammenführen
cohort.dataset.bielefeld <- cohort.patients.bielefeld %>% left_join(tmp, by = "PATIENT")
head(cohort.dataset.bielefeld)

### Generierung lokal aggregierter Zwischenergebnisse für die Merkmale

In [ ]:
# Lokal aggregierte Zwischenergebnisse für Bielefeld generieren
cohort.localresult.bielefeld <- cohort.dataset.bielefeld %>%
    group_by(VITALSTATUS) %>%
    summarize(OBESITY = sum(OBESITY, na.rm=TRUE), 
              FATIGUE = sum(FATIGUE, na.rm=TRUE), 
              RESPIRATORY_DISTRESS = sum(RESPIRATORY_DISTRESS, na.rm=TRUE))
head(cohort.localresult.bielefeld)

### Chi-Quadrat-Test zwischen Vitalstatus & Nebendiagnosen durchführen

In [ ]:
# Ergebnis in Dataframe konvertieren und Vitalstatus zu Zeilennamen umformatieren
tmp <- as.data.frame(cohort.localresult.bielefeld)
rownames(tmp) = tmp$VITALSTATUS
tmp <- tmp[-1]
head(tmp)

# Chi-Quadrat-Test zur Abhängigkeit zwischen Vitalstatus und den Nebendiagnosen für Bielefeld berechnen
cs <- chisq.test(tmp)
cs
corrplot(cs$residuals, is.cor = FALSE)

## Mannheimer Daten laden & identisch aufbereiten

In [ ]:
# Daten Mannheim laden
patients.mannheim  <- read_csv(paste(base_url, "rohdaten/mannheim/csv/patients.csv",   sep=""), col_types = cols(BIRTHDATE = col_date(format = "%Y-%m-%d")))
diagnoses.mannheim <- read_csv(paste(base_url, "rohdaten/mannheim/csv/conditions.csv", sep=""), col_types = cols(START = col_date(format = "%Y-%m-%d"), STOP = col_date(format = "%Y-%m-%d")))

# Alter ergänzen
patients.mannheim <- mutate(patients.mannheim, AGE = case_when(is.na(DEATHDATE) ~ as.period(interval(start = BIRTHDATE, end = Sys.Date()))$year, TRUE ~ as.period(interval(start = BIRTHDATE, end = DEATHDATE))$year))

# Patienten mit COVID-19 selektieren
cohort.patients.mannheim <- diagnoses.mannheim %>% 
    filter(DESCRIPTION == "COVID-19") %>% 
    select(PATIENT) %>% distinct()

# Demographische Angaben ergänzen
cohort.patients.mannheim <- cohort.patients.mannheim %>% 
    inner_join(patients.mannheim, by=c("PATIENT" = "Id")) %>%
    select(PATIENT, BIRTHDATE, DEATHDATE, GENDER, AGE, CITY)

# Vitalstatus ergänzen
cohort.patients.mannheim <- mutate(cohort.patients.mannheim, VITALSTATUS = case_when(is.na(DEATHDATE) ~ "ALIVE", TRUE ~ "DEAD"))

# Begleitdiagnosen bestimmen
cohort.secondary_diagnoses.mannheim <- cohort.patients.mannheim %>% 
            inner_join(diagnoses.mannheim, by="PATIENT") %>%
            filter(!(DESCRIPTION %in% c("COVID-19", "Suspected COVID-19")))

# Zu untersuchende Nebendiagnosen (Adipositas, Erschöpfung und Atemnot) auslesen
tmp <- cohort.secondary_diagnoses.mannheim %>% 
       filter(DESCRIPTION %in% c("Body mass index 30+ - obesity (finding)","Fatigue (finding)","Respiratory distress (finding)")) %>%
       select(PATIENT, DESCRIPTION) %>%
       mutate(DIAGNOSISPRESENT = 1)

# Diagnosen zu Spalten pivotieren
tmp <- pivot_wider(tmp, names_from=DESCRIPTION, values_from=DIAGNOSISPRESENT, values_fill=NA)

# Spalten umbenennen
colnames(tmp) <- c("PATIENT", "OBESITY", "FATIGUE", "RESPIRATORY_DISTRESS")

# Stammdaten & zu untersuchende Diagnosen zusammenführen
cohort.dataset.mannheim <- cohort.patients.mannheim %>% left_join(tmp, by = "PATIENT")
head(cohort.dataset.mannheim)

### Aggregiertes lokales Zwischenergebnis für Mannheim generieren

In [ ]:
# Lokal aggregierte Zwischenergebnisse für Mannheim generieren
cohort.localresult.mannheim <- cohort.dataset.mannheim %>%
    group_by(VITALSTATUS) %>%
    summarize(OBESITY = sum(OBESITY, na.rm=TRUE), 
              FATIGUE = sum(FATIGUE, na.rm=TRUE), 
              RESPIRATORY_DISTRESS = sum(RESPIRATORY_DISTRESS, na.rm=TRUE))
head(cohort.localresult.mannheim)

### Chi-Quadrat-Test für Mannheimer-Daten durchführen

In [ ]:
# Ergebnis in Dataframe konvertieren und Vitalstatus zu Zeilennamen umformatieren
tmp <- as.data.frame(cohort.localresult.mannheim)
rownames(tmp) = tmp$VITALSTATUS
tmp <- tmp[-1]

# Chi-Quadrat-Test zur Abhängigkeit zwischen Vitalstatus und den Nebendiagnosen für Mannheim berechnen
cs <- chisq.test(tmp)
cs
corrplot(cs$residuals, is.cor = FALSE)

# Ihre Aufgabe: kopieren Sie die Zellen für die Aufbereitung der Mannheimer Daten, um auch die Leipziger Daten identisch aufzubereiten!